In [4]:
import azurescript as azs

azs.ConnectToAzure()
azs.UploadData()
azs.CreateAzureDataset()
azs.PrepareAzureScript()
# azs.RunAzureScript()
print('done')

Workspace: AzureML-Pytorch
Compute Target: AzPytrch-NC6
Datastore: workspaceblobstore
Experiment: 616_Final
Uploading an estimated of 5 files
Uploading ./data\alldonesmall.csv
Uploaded ./data\alldonesmall.csv, 1 files out of an estimated total of 5
Uploading ./data\alldone.csv
Uploaded ./data\alldone.csv, 2 files out of an estimated total of 5
Uploading ./data\test_dl.pt
Uploaded ./data\test_dl.pt, 3 files out of an estimated total of 5
Uploading ./data\Payment_Output.csv
Uploaded ./data\Payment_Output.csv, 4 files out of an estimated total of 5
Uploading ./data\train_dl.pt
Uploaded ./data\train_dl.pt, 5 files out of an estimated total of 5
Uploaded 5 files
[({
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-37cc9333-cffe-4a2a-ba89-7ed5c59dd573",
  "account_name": "azuremlpytorch4611169748",
  "protocol": "https",
  "endpoint": "core.windows.net"
}, 'data')]
done


In [13]:
## LOCAL TEST MODEL
import torch
import scripts.model as mod
import scripts.preprocess as pp
import torch.nn as nn
from torch import optim
from torch.nn.modules.activation import Sigmoid


loss_func = nn.CrossEntropyLoss()
lr = .1
model = mod.SimpleMLP()
train_dl, test_dl = pp.LoadPreProcess()
opt = optim.SGD(model.parameters(), lr=lr)
epochs = 10

for epoch in range(epochs):

    model.train()
    for xb, yb in train_dl:
        # print(xb.float())
        # print(type(xb))
        pred = model(xb.float())
        # print(len(pred))
        # print(type(pred))
        # print(len(pred[0]))
        loss = loss_func(pred, yb.long())

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    val_loss = 0
    last_val_loss = 0
    num = 0
    num_correct = 0
    with torch.no_grad():
        for xb, yb in test_dl:
            pred = model(xb.float())
            val_loss += loss_func(pred, yb.long())
            pred[pred<.5]==0
            pred[pred>=.5]==1
            num_correct += (pred==yb).sum()
            num = len(yb)

    print('Loss: ', epoch, val_loss)
    print('Accuraccy: ,', num_correct / num)
    if last_val_loss > val_loss: print('OVERTRAINING')
    last_val_loss = val_loss



RuntimeError: The size of tensor a (484) must match the size of tensor b (100) at non-singleton dimension 1

In [1]:
##LOCAL TEST PREPRPOCESS

import scripts.preprocess as pp
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
datapath = "./data/"
master, payments = pp.LoadData(datapath, filename='alldone.csv')
print('Data Loaded')
solution, master = pp.PrepMLPData(master, payments)
print('Solution String Created')
train_dl, test_dl = pp.CreateMLPDataLoader(master, solution)
print('Data Loader Created')

filename = os.path.join(datapath, 'train_dl.pt')
torch.save(train_dl, 'train_dl.pt')
filename = os.path.join(datapath, 'test_dl.pt')
torch.save(test_dl, 'test_dl.pt')
print('YAY!')

Data Loaded
Solution String Created
Data Loader Created
YAY!


In [5]:
n = 0
for x, y in train_dl:
    print(x)
    n += 1
    if n > 1: break

tensor([[8.0000e+00, 2.1101e+05, 3.0857e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 2.2596e+05, 3.2309e+05,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [8.0000e+00, 1.8101e+05, 2.7874e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [8.0000e+00, 1.8083e+05, 2.7855e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 1.9975e+05, 2.9737e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 2.4305e+05, 3.4004e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], dtype=torch.float64)
tensor([[8.0000e+00, 1.9525e+05, 2.9291e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 1.9755e+05, 2.9520e+05,  ..., 1.0000e+00, 0.0000e+00,
         0.0000e+00],
        [8.0000e+00, 2.2625e+05, 3.2337e+05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [8.0000e+00, 2.1404e+05, 3.1160e+05,  ..., 0.0000e+00, 0.0000e+00,
    

In [15]:
master[master['accountid']==24837]

,clientname,accountid,customerid,ssnlastfour,Address1,Address2,City,State,PostalCode,dateplaced,...,placeddpd,hasbrokenpromise,hasconfirmed,haskeptptp,haspayments,hasplacementphone,haspossiblenumber,haspromever,Month1,Month2
